In [54]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [55]:
train_data = pd.read_csv('data/augmented_data_with_gpt2.csv')
val_data = pd.read_csv('data/preprocessed_val_data.csv')

In [56]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,903653690,Create HPA Behavior E2E Test,#### What would you like to be added:\r\n\r\nW...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/e2e-test-framework']",111253.0,['test/e2e/autoscaling/horizontal_pod_autoscal...,"['modified', 'modified']","[144, 43]","[16, 6]","[160, 49]",/sig autoscaling\r\n/triage accepted\r\n/help ...,create hpa behavior e2e test like add need e2e...,False
1,https://api.github.com/repos/kubernetes/kubern...,388343406,api timeouts when using token authentication,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],71816.0,"['cmd/kube-apiserver/app/server.go', 'pkg/cont...","['modified', 'modified', 'modified', 'modified...","[6, 16, 2, 35, 2, 34, 10]","[1, 3, 0, 1, 0, 1, 3]","[7, 19, 2, 36, 2, 35, 13]",@kubernetes/sig-auth-bugs @jmcmeek: Reiteratin...,api timeout token authentication use template ...,False
2,https://api.github.com/repos/kubernetes/kubern...,791563924,Update in-tree PV kubernetes topology label to GA,\r\n**What happened**:\r\nThe current in-tree ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],101534.0,"['pkg/volume/azuredd/azure_provision.go', 'sta...","['modified', 'modified', 'modified', 'modified...","[4, 5, 20, 2, 3, 5]","[4, 0, 11, 2, 3, 4]","[8, 5, 31, 4, 6, 9]",/assign @kassarl @Jiawei0227: GitHub didn't al...,update tree pv kubernete topology label ga hap...,False
3,https://api.github.com/repos/kubernetes/enhanc...,1268994650,Kubelet Evented PLEG for Better Performance,### Enhancement Description\r\n\r\n- One-line ...,https://github.com/kubernetes/enhancements/iss...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],115967.0,"['pkg/features/kube_features.go', 'pkg/kubelet...","['modified', 'modified', 'modified', 'modified']","[2, 5, 41, 7]","[1, 0, 0, 0]","[3, 5, 41, 7]","/sig-node Hello @harche 👋, 1.25 Enhancements ...",kubelet evente pleg well performance enhanceme...,False
4,https://api.github.com/repos/kubernetes/kubern...,331786462,SharedInformer documentation is missing or har...,<!-- This form is for bug reports and feature ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],112200.0,['staging/src/k8s.io/apiextensions-apiserver/e...,"['modified', 'modified', 'modified', 'modified...","[75, 75, 75, 83, 75, 75, 75, 75, 75, 75, 75, 75]","[4, 4, 4, 11, 4, 4, 4, 4, 4, 4, 4, 4]","[79, 79, 79, 94, 79, 79, 79, 79, 79, 79, 79, 79]",/cc @kubernetes/sig-api-machinery-bugs /kind d...,sharedinformer documentation miss hard find fo...,False


In [57]:
# check for null values
train_data.isnull().sum()


comments_url    2567
id                 0
title              0
body               0
issue_url       2567
pr_url          2567
labels             0
pr_number       2567
filename        2567
status          2567
additions       2567
deletions       2567
changes         2567
all_comments    2582
all_text           0
is_augmented       0
dtype: int64

In [58]:
val_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,859339139,Add sync to the windows node for storage E2E t...,#### What would you like to be added:\r\n\r\nW...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/test'],103976,['test/images/agnhost/Dockerfile_windows'],['modified'],[4],[0],[4],/sig testing /sig storage\r\n/kind flake /sig ...,add sync window node storage e2e test like add...
1,https://api.github.com/repos/kubernetes/kubern...,640544731,Scheduling framework: test more failure points...,See https://github.com/kubernetes/kubernetes/b...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/provider/gcp']",103642,['test/integration/scheduler/framework_test.go'],['modified'],[394],[122],[516],/sig scheduling Issues go stale after 90d of i...,scheduling framework test failure point trigge...
2,https://api.github.com/repos/kubernetes/kubern...,1605661093,kmsv2: collect metrics and log when a resource...,Metric:\r\nToday we have the `transformation_o...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],119166,['staging/src/k8s.io/apiserver/pkg/storage/val...,"['modified', 'modified', 'modified', 'modified']","[1, 5, 33, 71]","[1, 5, 0, 6]","[2, 10, 33, 77]",/triage accepted\r\n/sig auth @ritazh I would ...,kmsv2 collect metric log resource decryption f...
3,https://api.github.com/repos/kubernetes/kubern...,540032355,Move ImageLocality priority logic to its Score...,/sig scheduling\r\n/priority important-soon\r\...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/conformance'],86548,"['pkg/scheduler/algorithm/priorities/BUILD', '...","['modified', 'removed', 'removed', 'modified',...","[0, 0, 0, 1, 2, 69, 27]","[4, 109, 233, 1, 6, 5, 19]","[4, 109, 233, 2, 8, 74, 46]",/unassign /assign Note that this shouldn't nee...,imagelocality priority logic score plugin migr...
4,https://api.github.com/repos/kubernetes/kubern...,1808395080,[StructuredAuthenticationConfig] Rename `Clien...,Just a thought that came up as I...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],118984,"['pkg/kubeapiserver/authenticator/config.go', ...","['modified', 'modified', 'modified', 'modified...","[21, 65, 260, 41, 110, 210, 131, 204, 414, 131...","[67, 9, 21, 0, 0, 0, 0, 0, 0, 0, 84, 192, 0]","[88, 74, 281, 41, 110, 210, 131, 204, 414, 131...",/sig auth\r\n/triage accepted\r\n/assign,structuredauthenticationconfig rename clientid...


In [59]:
val_data.isnull().sum()

comments_url    0
id              0
title           0
body            0
issue_url       0
pr_url          0
labels          0
pr_number       0
filename        0
status          0
additions       0
deletions       0
changes         0
all_comments    5
all_text        0
dtype: int64

In [60]:
# remove all_text that have null values
train_data = train_data[train_data['all_text'].notnull()]
val_data = val_data[val_data['all_text'].notnull()]

# remove all_text that has value 'NaN ' or 'nan'
train_data = train_data[~train_data['all_text'].isin(['NaN ', 'nan'])]
val_data = val_data[~val_data['all_text'].isin(['NaN ', 'nan'])]

Check the label distribution

In [61]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/test: 271
2. area/e2e-test-framework: 203
3. area/apiserver: 203
4. area/cloudprovider: 203
5. area/kubelet: 203
6. area/code-generation: 203
7. area/dependency: 203
8. area/kubectl: 203
9. area/provider/gcp: 203
10. area/kubeadm: 203
11. area/ipvs: 203
12. area/provider/azure: 203
13. area/kube-proxy: 203
14. area/conformance: 203
15. area/custom-resources: 203
16. area/release-eng: 203


In [62]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in val_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Validation Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Validation Data):
1. area/test: 70
2. area/apiserver: 22
3. area/kubectl: 22
4. area/kubelet: 22
5. area/e2e-test-framework: 18
6. area/kubeadm: 12
7. area/provider/gcp: 4
8. area/cloudprovider: 4
9. area/dependency: 4
10. area/code-generation: 4
11. area/conformance: 2
12. area/provider/azure: 2
13. area/kube-proxy: 2
14. area/release-eng: 2
15. area/custom-resources: 1


Stopwords Removal

In [63]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

In [64]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Data Lemmatization

In [65]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [66]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


# Export the Data

In [67]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,903653690,Create HPA Behavior E2E Test,#### What would you like to be added:\r\n\r\nW...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/e2e-test-framework']",111253.0,['test/e2e/autoscaling/horizontal_pod_autoscal...,"['modified', 'modified']","[144, 43]","[16, 6]","[160, 49]",/sig autoscaling\r\n/triage accepted\r\n/help ...,create hpa behavior e2e test like add need e2e...,False
1,https://api.github.com/repos/kubernetes/kubern...,388343406,api timeouts when using token authentication,<!-- Please use this template while reporting ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/apiserver'],71816.0,"['cmd/kube-apiserver/app/server.go', 'pkg/cont...","['modified', 'modified', 'modified', 'modified...","[6, 16, 2, 35, 2, 34, 10]","[1, 3, 0, 1, 0, 1, 3]","[7, 19, 2, 36, 2, 35, 13]",@kubernetes/sig-auth-bugs @jmcmeek: Reiteratin...,api timeout token authentication use template ...,False
2,https://api.github.com/repos/kubernetes/kubern...,791563924,Update in-tree PV kubernetes topology label to GA,\r\n**What happened**:\r\nThe current in-tree ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],101534.0,"['pkg/volume/azuredd/azure_provision.go', 'sta...","['modified', 'modified', 'modified', 'modified...","[4, 5, 20, 2, 3, 5]","[4, 0, 11, 2, 3, 4]","[8, 5, 31, 4, 6, 9]",/assign @kassarl @Jiawei0227: GitHub didn't al...,update tree pv kubernete topology label ga hap...,False
3,https://api.github.com/repos/kubernetes/enhanc...,1268994650,Kubelet Evented PLEG for Better Performance,### Enhancement Description\r\n\r\n- One-line ...,https://github.com/kubernetes/enhancements/iss...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],115967.0,"['pkg/features/kube_features.go', 'pkg/kubelet...","['modified', 'modified', 'modified', 'modified']","[2, 5, 41, 7]","[1, 0, 0, 0]","[3, 5, 41, 7]","/sig-node Hello @harche 👋, 1.25 Enhancements ...",kubelet evente pleg performance enhancement de...,False
4,https://api.github.com/repos/kubernetes/kubern...,331786462,SharedInformer documentation is missing or har...,<!-- This form is for bug reports and feature ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],112200.0,['staging/src/k8s.io/apiextensions-apiserver/e...,"['modified', 'modified', 'modified', 'modified...","[75, 75, 75, 83, 75, 75, 75, 75, 75, 75, 75, 75]","[4, 4, 4, 11, 4, 4, 4, 4, 4, 4, 4, 4]","[79, 79, 79, 94, 79, 79, 79, 79, 79, 79, 79, 79]",/cc @kubernetes/sig-api-machinery-bugs /kind d...,sharedinformer documentation miss hard find fo...,False


In [68]:
# count nan values in the train_data
train_data.isna().sum()

comments_url    2567
id                 0
title              0
body               0
issue_url       2567
pr_url          2567
labels             0
pr_number       2567
filename        2567
status          2567
additions       2567
deletions       2567
changes         2567
all_comments    2582
all_text           0
is_augmented       0
dtype: int64

In [69]:
# export the train_data and val_data to csv
train_data.to_csv('final_preprocessed_train_data.csv', index=False)
val_data.to_csv('final_preprocessed_val_data.csv', index=False)